# 📊 Análise de Resultados — PySpark + Delta Lake

Este notebook tem como objetivo realizar análises exploratórias e agregações a partir dos dados estruturados na camada **Trusted** do Data Warehouse, utilizando **PySpark** em conjunto com o **Delta Lake**. Serão exploradas informações provenientes de tabelas fato e dimensões, permitindo insights sobre o perfil das empresas, distribuição geográfica, porte, natureza jurídica, entre outros aspectos relevantes.

## Configurações e importes

In [1]:
# Para iniciar a seção spark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DecimalType
from pyspark.sql import functions as f
from datetime import datetime
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable

In [2]:
builder = SparkSession.builder \
    .appName("App analise de dados") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g")

spark: SparkSession = configure_spark_with_delta_pip(builder).getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/22 09:35:25 WARN Utils: Your hostname, BRALSOFT42, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/22 09:35:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/wilcb/projeto_data_warehouse/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/wilcb/.ivy2.5.2/cache
The jars for the packages stored in: /home/wilcb/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ed25fd59-7190-4c1f-8fe2-77597577bde3;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 364ms :: artifacts dl 12ms
	:: mod

## 📊 Análise de Resultados

### Iniciar DeltaSpark

In [3]:
try:    
    deltaTable: DeltaTable = DeltaTable.forPath(spark, "../TRS/fato_cnpj")
    df: DataFrame = deltaTable.toDF()
    df.show(5, truncate=False)
    df.printSchema()
except Exception as e:
    print(f"Erro na leitura: {e}")

25/07/22 09:36:09 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+----------+-------+-------------+-------------------------------+------------------+-----------------------+-------------------------+--------------------+-----------+---------------------+-----------+----------------------+---------------+---------------------+------+-----------+------+--------+---+----------------+----+---------+----+---------+-------+--------+-------------------------+-----------------+----------------------+----------------------------------------+-----------------+------------+------------------------+-------------------------------+--------------+--------------------+---------------------------+---------------------------+-------------+------------------+---------------------+---------+--------------+-----------------+
|cnpj_base|cnpj_ordem|cnpj_dv|matriz_filial|nome_fantasia                  |situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_cidade_exterior|codigo_pais|data_inicio_atividade|cnae_fiscal|cnae_fiscal_secundaria|ti

### Visão Geral

In [4]:
df.printSchema()

root
 |-- cnpj_base: string (nullable = true)
 |-- cnpj_ordem: string (nullable = true)
 |-- cnpj_dv: string (nullable = true)
 |-- matriz_filial: string (nullable = true)
 |-- nome_fantasia: string (nullable = true)
 |-- situacao_cadastral: string (nullable = true)
 |-- data_situacao_cadastral: date (nullable = true)
 |-- motivo_situacao_cadastral: string (nullable = true)
 |-- nome_cidade_exterior: string (nullable = true)
 |-- codigo_pais: string (nullable = true)
 |-- data_inicio_atividade: date (nullable = true)
 |-- cnae_fiscal: string (nullable = true)
 |-- cnae_fiscal_secundaria: string (nullable = true)
 |-- tipo_logradouro: string (nullable = true)
 |-- logradouro: string (nullable = true)
 |-- numero: string (nullable = true)
 |-- complemento: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- cep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- codigo_municipio: string (nullable = true)
 |-- ddd1: string (nullable = true)
 |-- telefone1: str

In [5]:
df_formatado = df.select(
    f.concat(
        df.cnpj_base,
        df.cnpj_ordem,
        df.cnpj_dv
    ).alias("CNPJ"),
    df["nome_fantasia"].alias("NOME FANTASIA"),
    df["situacao_cadastral"].alias("SITUAÇÃO CADASTRAL"),
    f.date_format(df["data_situacao_cadastral"], "dd/MM/yyyy").alias("DATA SITUAÇÃO CADASTRAL"),
    df["motivo_situacao_cadastral"].alias("SITUAÇÃO CADASTRAL MOTIVO"),
    df["nome_cidade_exterior"].alias("CIDADE EXTERIOR"),
    df["codigo_pais"].alias("CÓDIGO DO PAÍS"),
    f.date_format(df["data_inicio_atividade"], "dd/MM/yyyy").alias("DATA INÍCIO ATIVIDADE"),
    df["cnae_fiscal"].alias("CNAE PRINCIPAL"),
    df["cnae_fiscal_secundaria"].alias("CNAES SECUNDÁRIOS"),
    f.concat(
        df["tipo_logradouro"],
        f.lit(" "),
        df["logradouro"],
        f.lit(", "),
        df["numero"]
    ).alias("ENDEREÇO"),
    df["complemento"].alias("COMPLEMENTO"),
    df["bairro"].alias("BAIRRO"),
    df["cep"].alias("CEP"),
    df["municipio"].alias("MUNICÍPIO"),
    df["uf"].alias("UF"),
    df["telefone1"].alias("TELEFONE"),
    df["email"].alias("EMAIL"),
    df["descricao_cnae"].alias("DESCRIÇÃO CNAE"),
    df["capital_social"].alias("CAPITAL SOCIAL"),
    df["razao_social"].alias("RAZÃO SOCIAL"),
    df["codigo_natureza_juridica"].alias("CÓDIGO NATUREZA JURÍDICA"),
    df["descricao_natureza_juridica"].alias("DESCRIÇÃO NATUREZA JURÍDICA"),
    df["codigo_porte_empresa"].alias("CÓDIGO PORTE"),
    df["opcao_simples"].alias("OPÇÃO PELO SIMPLES"),
    f.date_format(df["data_opcao_simples"], "dd/MM/yyyy").alias("DATA OPÇÃO SIMPLES"),
    f.date_format(df["data_exclusao_simples"], "dd/MM/yyyy").alias("DATA EXCLUSÃO SIMPLES"),
    df["opcao_mei"].alias("OPÇÃO MEI"),
    f.date_format(df["data_opcao_mei"], "dd/MM/yyyy").alias("DATA OPÇÃO MEI"),
    f.date_format(df["data_exclusao_mei"], "dd/MM/yyyy").alias("DATA EXCLUSÃO MEI")
)
df_formatado.show(2, truncate=False)
df_formatado.printSchema()

+--------------+-------------------------------+------------------+-----------------------+-------------------------+---------------+--------------+---------------------+--------------+-----------------+-----------------------------+-----------+------+--------+---------------+---+--------+-----------------+----------------------------------------+--------------+------------+------------------------+---------------------------+------------+------------------+------------------+---------------------+---------+--------------+-----------------+
|CNPJ          |NOME FANTASIA                  |SITUAÇÃO CADASTRAL|DATA SITUAÇÃO CADASTRAL|SITUAÇÃO CADASTRAL MOTIVO|CIDADE EXTERIOR|CÓDIGO DO PAÍS|DATA INÍCIO ATIVIDADE|CNAE PRINCIPAL|CNAES SECUNDÁRIOS|ENDEREÇO                     |COMPLEMENTO|BAIRRO|CEP     |MUNICÍPIO      |UF |TELEFONE|EMAIL            |DESCRIÇÃO CNAE                          |CAPITAL SOCIAL|RAZÃO SOCIAL|CÓDIGO NATUREZA JURÍDICA|DESCRIÇÃO NATUREZA JURÍDICA|CÓDIGO PORTE|OPÇÃO PELO

### Análise de Abertura e Atividades das Empresas

Quantas empresas foram abertas por mês/ano

In [6]:
df.groupBy(
    f.year("data_inicio_atividade").alias("ANO"),
    f.month("data_inicio_atividade").alias("MÊS")
    ).agg(
        f.count("*").alias("Empresas abertas por mês/ano")
    ).orderBy("Empresas abertas por mês/ano", ascending=False) \
    .select("Empresas abertas por mês/ano".upper(), "MÊS", "ANO") \
    .show(10,truncate=False)

+----------------------------+---+----+
|EMPRESAS ABERTAS POR MÊS/ANO|MÊS|ANO |
+----------------------------+---+----+
|82291                       |8  |2024|
|65879                       |7  |2012|
|58368                       |1  |2025|
|49062                       |7  |2008|
|48040                       |7  |2004|
|45271                       |2  |2025|
|42544                       |5  |2025|
|42035                       |3  |2025|
|41942                       |7  |2024|
|41477                       |4  |2025|
+----------------------------+---+----+
only showing top 10 rows


In [8]:
status_inativa = ('02', '03', '04')

df_status_empresa = df.withColumn(
    "STATUS EMPRESA",
    f.when(
        df.situacao_cadastral == f.lit('01'),
        "ATIVA"
    ).when(
        df.situacao_cadastral.isin('02', '03', '04'),
        "INATIVA"
    ).otherwise("OUTROS")
)

# df_status_empresa.select("STATUS EMPRESA").show()
df_status_empresa.groupBy("STATUS EMPRESA").agg(
    f.count("*").alias("Qtd")
).show()

df_status_empresa.groupBy("STATUS EMPRESA").agg(
    f.count("*").alias("Qtd")
).describe("Qtd").show()

+--------------+-------+
|STATUS EMPRESA|    Qtd|
+--------------+-------+
|       INATIVA|2484499|
|        OUTROS|2260668|
|         ATIVA|   8268|
+--------------+-------+



+-------+------------------+
|summary|               Qtd|
+-------+------------------+
|  count|                 3|
|   mean|1584478.3333333333|
| stddev|1369618.3192117186|
|    min|              8268|
|    max|           2484499|
+-------+------------------+



### 📊 Estatísticas Gerais

#### Listar UFs distintas

In [9]:
df_formatado.select("UF").distinct().show()

+---+
| UF|
+---+
| SP|
| MG|
| RJ|
| PR|
| RS|
| BA|
| SC|
| GO|
| PE|
| CE|
| MT|
| ES|
| PA|
| DF|
| MA|
| PB|
| MS|
| RN|
| AM|
| PI|
+---+
only showing top 20 rows


#### Contagem de empresas por UF

In [22]:
df_contagem_uf_empresas = \
    df_formatado.groupBy("UF") \
    .agg(
        f.count("*").alias("Qtd Empresas")
    ).orderBy("Qtd Empresas", ascending=False)

df_contagem_uf_empresas.show()

+---+------------+
| UF|Qtd Empresas|
+---+------------+
| SP|     1379898|
| MG|      513667|
| RJ|      382888|
| RS|      344456|
| PR|      326165|
| BA|      237630|
| SC|      231308|
| GO|      169298|
| PE|      141706|
| CE|      134480|
| ES|       96212|
| PA|       93181|
| MT|       91885|
| DF|       79580|
| MA|       72871|
| MS|       63846|
| PB|       59851|
| RN|       53391|
| AM|       51203|
| AL|       45296|
+---+------------+
only showing top 20 rows


#### Decrição estatística

In [29]:
# Para mais de uma coluna -> df.describe(["col1", "col2"]).show()
df_formatado.describe("CAPITAL SOCIAL").show()

+-------+------------------+
|summary|    CAPITAL SOCIAL|
+-------+------------------+
|  count|            486655|
|   mean|   11290841.310325|
| stddev|7.89383137213955E8|
|    min|              0.00|
|    max|   500000352182.73|
+-------+------------------+



### 🔍 3. Consultas Relevantes para Insights

#### Top 10 municípios com mais empresas

In [10]:
df_municipio = df_formatado
df_municipio_por_uf = df_municipio.groupBy("MUNICÍPIO", "UF").agg(
    f.count("*").alias("Qtd Empresas"),
).orderBy("Qtd Empresas", ascending=False)
df_municipio_por_uf.show(10)

+--------------+---+------------+
|     MUNICÍPIO| UF|Qtd Empresas|
+--------------+---+------------+
|     SAO PAULO| SP|      430363|
|RIO DE JANEIRO| RJ|      168441|
|BELO HORIZONTE| MG|       91174|
|      BRASILIA| DF|       79580|
|      CURITIBA| PR|       75285|
|      SALVADOR| BA|       66713|
|  PORTO ALEGRE| RS|       62352|
|     FORTALEZA| CE|       62262|
|       GOIANIA| GO|       51450|
|        RECIFE| PE|       42108|
+--------------+---+------------+
only showing top 10 rows


#### Distribuição por Natureza Jurídica

In [ ]:
df_empresas_por_nj = df_formatado.groupBy("DESCRIÇÃO NATUREZA JURÍDICA","CÓDIGO NATUREZA JURÍDICA").agg(
    f.count("*").alias("Qtd empresas"),
).filter(df_formatado["CÓDIGO NATUREZA JURÍDICA"].isNotNull()).orderBy("Qtd empresas", ascending=False)
df_empresas_por_nj.show(truncate=False)

+------------------------------------------------------------------------+------------------------+------------+
|DESCRIÇÃO NATUREZA JURÍDICA                                             |CÓDIGO NATUREZA JURÍDICA|Qtd empresas|
+------------------------------------------------------------------------+------------------------+------------+
|Sociedade Empresária Limitada                                           |2062                    |182094      |
|Empresário (Individual)                                                 |2135                    |156222      |
|Candidato a Cargo Político Eletivo                                      |4090                    |44499       |
|Produtor Rural (Pessoa Física)                                          |4120                    |33767       |
|Associação Privada                                                      |3999                    |29262       |
|Sociedade Simples Limitada                                              |2240                  

## Delta Table - Controle de versões

In [ ]:
# Tabelas Trusted
deltaTable_estabelecimentos: DeltaTable = DeltaTable.forPath(spark, "../TRS/estabelecimentos")
deltaTable_empresas: DeltaTable = DeltaTable.forPath(spark, "../TRS/empresas")
deltaTable_municipios_rf: DeltaTable = DeltaTable.forPath(spark, "../TRS/municipios_rf")
deltaTable_cnae: DeltaTable = DeltaTable.forPath(spark, "../TRS/cnae")
deltaTable_natureza_juridica: DeltaTable = DeltaTable.forPath(spark, "../TRS/natureza_juridica")
deltaTable_simples_nacional: DeltaTable = DeltaTable.forPath(spark, "../TRS/simples_nacional")

# Tabela Fato
deltaTable.history().show(vertical=True, truncate=False)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------
 version             | 0                                                                                                                     
 timestamp           | 2025-07-20 15:54:41.123                                                                                               
 userId              | NULL                                                                                                                  
 userName            | NULL                                                                                                                  
 operation           | WRITE                                                                                                                 
 operationParameters | {mode -> Overwrite, partitionBy -> ["uf"]}                                                                            
 job  

In [51]:
def titulo(texto):
    print(f"{"=-"*20} >> {texto} << {"=-"*20}")

titulo("HISTÓRICO ESTABELECIMENTOS")
deltaTable_estabelecimentos.history().show(vertical=True, truncate=False)

titulo("HISTÓRICO EMPRESAS")
deltaTable_empresas.history().show(vertical=True, truncate=False)

titulo("HISTÓRICO MUNICÍPIOS RECEITA FEDERAL")
deltaTable_municipios_rf.history().show(vertical=True, truncate=False)

titulo("HISTÓRICO CNAE")
deltaTable_cnae.history().show(vertical=True, truncate=False)

titulo("HISTÓRICO NATUREZA JURÍDICA")
deltaTable_natureza_juridica.history().show(vertical=True, truncate=False)

titulo("HISTÓRICO SIMPLES NACIONAL")
deltaTable_simples_nacional.history().show(vertical=True, truncate=False)

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=- >> HISTÓRICO ESTABELECIMENTOS << =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 17                                                                                                                                                                                                                                                                                                                                                        
 timestamp           | 2025-07-20 14:31:32.288                                                                                                   

## Finalizar spark

In [112]:
spark.stop()